### **Download Sentiment Dataset**

In [ ]:
! pip install --quiet datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 59.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which i

In [ ]:
from datasets import load_dataset

sentiment_dataset = load_dataset("rasyosef/amharic-sentiment")
sentiment_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Generating train split:   0%|          | 0/2223 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/279 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 2223
    })
    dev: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 279
    })
    test: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 279
    })
})

In [ ]:
sentiment_dataset["train"].features

{'clean_tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [ ]:
label_names = sentiment_dataset["train"].features["label"].names
label_names

['negative', 'positive']

### **Tokenize Tweets**

In [ ]:
from transformers import AutoTokenizer

model_id = "rasyosef/bert-tiny-amharic"
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(tokenizer.tokenize("ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ ምዕተ [MASK] ተቆጥሯል።"))

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/274k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/725k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

['ከሀ', '##ገራቸው', 'ከኢትዮጵያ', 'ከወጡ', 'ግማሽ', 'ምዕተ', '[MASK]', 'ተቆጥ', '##ሯል', '።']


In [ ]:
def tokenize_dataset(samples):
  tokenized_samples = tokenizer(samples["clean_tweet"], truncation=True, max_length=512)
  return tokenized_samples

tokenize_dataset({"clean_tweet":["ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ"]})

{'input_ids': [[2, 12051, 11275, 10894, 27845, 13971, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
preprocessed_datasets = sentiment_dataset.map(
    tokenize_dataset,
    batched=True,
    remove_columns=["clean_tweet"]
  )
preprocessed_datasets

Map:   0%|          | 0/2223 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2223
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 279
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 279
    })
})

### **Load Amharic BERT model**

In [ ]:
id2label={i: label for i, label in enumerate(label_names)}
label2id={label: i for i, label in enumerate(label_names)}

print(id2label, label2id)

{0: 'negative', 1: 'positive'} {'negative': 0, 'positive': 1}


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    # device_map="cuda"
)

config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rasyosef/bert-tiny-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Finetuning**

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [ ]:
from transformers import TrainingArguments

batch_size = 32
epochs = 8

training_args = TrainingArguments(
    output_dir=model_id+"-finetuned",
    learning_rate=2e-5,
    #lr_scheduler_type="linear",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    seed=16,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric1 = evaluate.load("accuracy")
  metric2 = evaluate.load("precision")
  metric3 = evaluate.load("recall")
  metric4 = evaluate.load("f1")

  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
  precision = metric2.compute(predictions=predictions, references=labels, average='macro')["precision"]
  recall = metric3.compute(predictions=predictions, references=labels, average='macro')["recall"]
  f1 = metric4.compute(predictions=predictions, references=labels, average='macro')["f1"]

  return {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f1": f1
  }

compute_metrics(([[1,0], [0,1]], [0,1]))

{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=preprocessed_datasets["train"],
    eval_dataset=preprocessed_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5404195189476013,
 'eval_accuracy': 0.7777777777777778,
 'eval_precision': 0.7782563025210084,
 'eval_recall': 0.7737984496124031,
 'eval_f1': 0.7750000000000001,
 'eval_runtime': 4.7799,
 'eval_samples_per_second': 58.369,
 'eval_steps_per_second': 1.883}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.535400,0.503102,0.792115,0.792910,0.788217,0.789516
2,0.491800,0.480452,0.802867,0.802419,0.800388,0.801140
3,0.471800,0.465323,0.802867,0.804295,0.798760,0.800240
4,0.452700,0.461865,0.802867,0.803538,0.799302,0.800561
5,0.437200,0.460122,0.813620,0.813091,0.811473,0.812111
6,0.428700,0.456195,0.817204,0.816935,0.814806,0.815603
7,0.415800,0.455368,0.813620,0.813555,0.810930,0.811858
8,0.420400,0.454510,0.813620,0.813555,0.810930,0.811858


TrainOutput(global_step=560, training_loss=0.45672028745923726, metrics={'train_runtime': 49.6448, 'train_samples_per_second': 358.225, 'train_steps_per_second': 11.28, 'total_flos': 2831530992060.0, 'train_loss': 0.45672028745923726, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4561949074268341,
 'eval_accuracy': 0.8172043010752689,
 'eval_precision': 0.8169354838709677,
 'eval_recall': 0.8148062015503876,
 'eval_f1': 0.8156029287889588,
 'eval_runtime': 4.8886,
 'eval_samples_per_second': 57.072,
 'eval_steps_per_second': 1.841,
 'epoch': 8.0}

### **Evaluate Model**

In [ ]:
import torch

num_samples = len(preprocessed_datasets["test"])
inputs = data_collator([preprocessed_datasets["test"][i] for i in range(num_samples)])
# print(inputs)

#output = model(**inputs)
output = model(**{k: v.to("cuda") for k, v in inputs.items()})
y_pred = np.argmax(output.logits.tolist(), axis=-1)
y_test = np.array(preprocessed_datasets["test"]["label"])

print(y_pred)
print(y_test)

[1 1 1 1 0 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1
 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 0 0 0
 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1
 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0
 0 0 1 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 1 1 0 1
 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0
 1 0 0 1 0 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0]
[1 1 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1
 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0
 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 0 1
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0
 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 1 0

In [ ]:
from sklearn import metrics

metrics.confusion_matrix(y_test, y_pred)

array([[124,  26],
       [ 33,  96]])

In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.83      0.81       150
           1       0.79      0.74      0.76       129

    accuracy                           0.79       279
   macro avg       0.79      0.79      0.79       279
weighted avg       0.79      0.79      0.79       279



### **Test in Pipeline**

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
pipe("ቆንጆ ፊልም ነው")

[{'label': 'positive', 'score': 0.8122960925102234}]

In [ ]:
pipe("አሪፍ ልብስ")

[{'label': 'negative', 'score': 0.5792895555496216}]

In [ ]:
pipe("ጅላንፎ")

[{'label': 'negative', 'score': 0.8795253038406372}]

In [ ]:
pipe("እዚህ ሰፈር በማታ መጓዝ እፈራለሁ")

[{'label': 'negative', 'score': 0.5435606837272644}]

In [ ]:
pipe("ዩክረን እና ሩስያ ከባድ ውግያ ላይ ናቸው")

[{'label': 'positive', 'score': 0.6604615449905396}]